Setting up Kubernetes (K8s) can vary depending on your environment (local development, cloud provider, etc.). Below is a step-by-step guide to help you get started with Kubernetes, whether you're deploying locally (e.g., Minikube) or on a cloud provider like **Google Kubernetes Engine (GKE)**.

---

### **Option 1: Local Development with Minikube**
Minikube runs a single-node Kubernetes cluster on your local machine for testing.

#### **1. Install Prerequisites**
- **Install Docker**: [Docker Desktop](https://www.docker.com/products/docker-desktop) (for containerization).
- **Install kubectl**: Kubernetes command-line tool.
  ```bash
  # For macOS (Homebrew):
  brew install kubectl

  # For Linux (Debian/Ubuntu):
  sudo apt-get update && sudo apt-get install -y kubectl
  ```
- **Install Minikube**:
  ```bash
  # macOS/Linux:
  brew install minikube

  # Linux (curl):
  curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
  sudo install minikube-linux-amd64 /usr/local/bin/minikube
  ```

#### **2. Start Minikube**
```bash
minikube start --driver=docker  # Use Docker as the VM driver
minikube status                # Verify cluster is running
```

#### **3. Deploy a Sample App**
- Create a `deployment.yaml`:
  ```yaml
  apiVersion: apps/v1
  kind: Deployment
  metadata:
    name: flask-app
  spec:
    replicas: 2
    selector:
      matchLabels:
        app: flask
    template:
      metadata:
        labels:
          app: flask
      spec:
        containers:
        - name: flask-container
          image: your-dockerhub-username/flask-app:latest
          ports:
          - containerPort: 5000
  ```
- Create a `service.yaml` to expose the app:
  ```yaml
  apiVersion: v1
  kind: Service
  metadata:
    name: flask-service
  spec:
    type: NodePort
    selector:
      app: flask
    ports:
      - protocol: TCP
        port: 80
        targetPort: 5000
  ```
- Apply the configurations:
  ```bash
  kubectl apply -f deployment.yaml
  kubectl apply -f service.yaml
  ```

#### **4. Access the App**
```bash
minikube service flask-service  # Opens the app in your browser
```

---

### **Option 2: Google Kubernetes Engine (GKE)**
For production-grade Kubernetes clusters on Google Cloud.

#### **0. Install Prerequisites**
- Install **gcloud CLI**: [Google Cloud SDK](https://cloud.google.com/sdk/docs/install).
- Authenticate and configure:
  ```bash
  gcloud auth login
  gcloud config set project YOUR_PROJECT_ID  
  ```

#### **1. Instale o componente kubectl**:
  ```bash
  gcloud components install kubectl
  ```

Para confirmar se kubectl está instalado, confira se ele tem a versão mais recente:
  ```bash
  kubectl version --client
  ```

- ##### Ver kubeconfig
    Para ver a kubeconfig do ambiente, execute o seguinte comando:

  ```bash
      kubectl config view
  ```

#### **2. Create a GKE Cluster**
```bash
gcloud container clusters create-auto flask-cluster --region us-central1 --project YOUR_PROJECT_ID
```

#### **3. Deploy Your Flask App**

- **Step 0: Write Your Flask App**

    Create a `app.py`:
    ```python
    import os
    from flask import Flask
    
    app = Flask(__name__)
    
    @app.route('/')
    def hello_world():
        target = os.environ.get('TARGET', 'World')
        return 'Hello {}!\n'.format(target)
    
    if __name__ == "__main__":
        app.run(debug=True,host='0.0.0.0',port=int(os.environ.get('PORT', 8080)))
    ```

- **Step 1: Dockerize Your Flask App**
  Create a `Dockerfile`:
  ```dockerfile
    FROM python:3.13-slim
    
    # Allow statements and log messages to immediately appear in the Knative logs
    ENV PYTHONUNBUFFERED True
    
    # Copy local code to the container image.
    ENV APP_HOME /app
    WORKDIR $APP_HOME
    COPY . ./
    
    # Install production dependencies.
    RUN pip install Flask gunicorn
        
    # Run the web service on container startup. Here we use the gunicorn
    # webserver, with one worker process and 8 threads.
    # For environments with multiple CPU cores, increase the number of workers
    # to be equal to the cores available.
    # CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 app:app
    CMD exec gunicorn --bind 0.0.0.0:${PORT:-5000} --workers 1 --threads 8 app:app
  ```

Build and push to **Google Container Registry (GCR)**:
  ```bash
  docker build -t gcr.io/YOUR_PROJECT_ID/flask-app:v1 .
  docker push gcr.io/YOUR_PROJECT_ID/flask-app:v1
  ```

- **Step 2: Create Kubernetes Manifests**
  - `deployment.yaml`:
    ```yaml
    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: flask-deployment
    spec:
      replicas: 3
      selector:
        matchLabels:
          app: flask
      template:
        metadata:
          labels:
            app: flask
        spec:
          containers:
          - name: flask
            image: gcr.io/YOUR_PROJECT_ID/flask-app:v1
            ports:
            - containerPort: 5000
            env:
            - name: PORT           # Explicitly set PORT
              value: "5000"        # Use 5000 or your app's default port
    ```

  - `service.yaml` (to expose the app externally):
    ```yaml
    apiVersion: v1
    kind: Service
    metadata:
      name: flask-service
    spec:
      type: LoadBalancer
      selector:
        app: flask
      ports:
        - protocol: TCP
          port: 80
          targetPort: 5000
    ```

- **Step 3: Deploy to GKE**
  ```bash
  kubectl apply -f deployment.yaml
  kubectl apply -f service.yaml
  ```

#### **4. Access the App**
Get the external IP:
```bash
kubectl get service flask-service
```
Visit `http://EXTERNAL_IP` in your browser.

---

### **Key Kubernetes Commands**
| Command                          | Description                                  |
|----------------------------------|----------------------------------------------|
| `kubectl get pods`               | List running pods                            |
| `kubectl describe pod POD_NAME`  | Debug pod issues                             |
| `kubectl logs POD_NAME`          | View pod logs                                |
| `kubectl delete -f file.yaml`    | Delete resources defined in a YAML file      |
| `kubectl scale deployment NAME --replicas=5` | Scale a deployment |

---

### **Scaling and Monitoring**
- **Autoscaling**:
  ```bash
  # Horizontal Pod Autoscaling (HPA)
  kubectl autoscale deployment flask-deployment --cpu-percent=50 --min=2 --max=10
  ```
- **Monitoring**:
  - Use **Google Cloud Monitoring** (built-in with GKE).
  - For Minikube, install the Kubernetes Dashboard:
    ```bash
    minikube dashboard
    ```

---

### **Cleanup**
- **GKE**:
  ```bash
  gcloud container clusters delete flask-cluster --region us-central1
  ```
- **Minikube**:
  ```bash
  minikube stop
  minikube delete
  ```

---

### **Troubleshooting Tips**
- **Image Pull Errors**: Ensure your Docker image is pushed to GCR/Docker Hub.
- **Port Conflicts**: Verify `targetPort` in the service matches the container port.
- **Resource Limits**: Add CPU/memory limits to your deployment YAML:
  ```yaml
  resources:
    limits:
      cpu: "500m"
      memory: "512Mi"
  ```

---

If you need help with **specific parts** (e.g., configuring Ingress, persistent storage, or CI/CD), let me know! 😊

# Troubleshoot

Your deployment is failing because the pods are crashing immediately after starting. Let’s diagnose and fix this step-by-step:

---

### **1. Pods in `CrashLoopBackOff`**
The **CrashLoopBackOff** status indicates your container starts but crashes repeatedly. Common causes:
- **Application errors** (e.g., Flask crashes on startup).
- **Missing dependencies** in the Docker image.
- **Incorrect environment variables** (e.g., `PORT` not set).

---

### **2. Debugging Steps**
#### **Check Pod Logs**
Forcefully fetch logs from the crashing pod (retry after a few minutes):
```bash
kubectl logs flask-deployment-55d5558cbc-8n7l9 --previous
```

#### **Inspect Resource Requests**
Your deployment YAML requests **500m CPU** and **2Gi memory**. If your app uses more than this, it will crash.  
Adjust the `resources` section in `deployment.yaml`:
```yaml
resources:
  requests:
    cpu: "250m"  # Reduce CPU request
    memory: "512Mi"  # Reduce memory request
  limits:
    cpu: "500m"
    memory: "1Gi"
```

#### **Verify the Docker Image**
Ensure the image `gcr.io/YOUR_PROJECT_ID/flask-app:v1` is correctly pushed to GCR and accessible:
```bash
gcloud container images list --repository=gcr.io/YOUR_PROJECT_ID/flask-app
```

#### **Test the Image Locally**
Run the Docker image locally to check for startup issues:
```bash
docker run -e PORT=5000 -p 5000:5000 gcr.io/YOUR_PROJECT_ID/flask-app:v1
```
If it crashes, debug your Flask app locally.

---

### **3. Common Fixes**
#### **Missing Dependencies**
Your Dockerfile installs `ffmpeg`, `libsm6`, and `libxext6` for OpenCV, but your app might need additional libraries. Add:
```dockerfile
RUN apt-get update && \
    apt-get install -y --no-install-recommends \
    libgl1 libglib2.0-0 && \  # Add OpenGL libraries
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*
```

#### **Environment Variables**
Ensure your Flask app reads the `PORT` environment variable:
```python
import os
port = int(os.environ.get("PORT", 5000))
app.run(host="0.0.0.0", port=port)
```

#### **Gunicorn Configuration**
Your current Gunicorn command (`--workers 1 --threads 80`) may cause resource exhaustion. Simplify:
```dockerfile
CMD exec gunicorn --bind :$PORT --workers 2 --timeout 0 main:app
```

---

### **4. Update and Redeploy**
1. Fix the Dockerfile and application code based on your local tests.
2. Rebuild and push the image:
   ```bash
   docker build -t gcr.io/YOUR_PROJECT_ID/flask-app:v2 .
   docker push gcr.io/YOUR_PROJECT_ID/flask-app:v2
   ```
3. Update the image tag in `deployment.yaml`:
   ```yaml
   image: gcr.io/YOUR_PROJECT_ID/flask-app:v2
   ```
4. Redeploy:
   ```bash
   kubectl delete -f deployment.yaml -f service.yaml
   kubectl apply -f deployment.yaml -f service.yaml
   ```

---

### **5. Verify the Fix**
Check pod status and logs:
```bash
kubectl get pods
kubectl logs <POD_NAME>
```
If the pods stabilize, access your app via the LoadBalancer’s `EXTERNAL-IP`:
```bash
kubectl get service flask-service  # Use the EXTERNAL-IP
```

---

### **Troubleshooting Summary**
| Symptom               | Likely Cause                | Fix                                  |
|-----------------------|-----------------------------|--------------------------------------|
| `CrashLoopBackOff`    | Flask app crash on startup  | Check logs, test image locally       |
| `Pending` pods        | Resource limits too high    | Reduce CPU/memory requests           |
| `Error` status        | Missing system dependencies | Add required libraries in Dockerfile |

If the issue persists, share the output of:
```bash
kubectl logs <POD_NAME> --previous
```